In [ ]:
###### test submission block
import time
start_time = time.time()

holidays=['2012-01-31','2012-01-16','2012-02-20','2012-05-28','2012-07-04','2012-09-03','2012-10-08','2012-11-12','2012-11-22','2012-12-25',
'2013-01-01','2013-01-21','2013-02-18','2013-05-27','2013-07-04','2013-09-02','2013-10-14','2013-11-11','2013-11-28','2013-12-25','2014-01-01',
'2014-01-20','2014-02-17','2014-05-26','2014-07-04','2014-09-01','2014-10-10','2014-11-11','2014-11-27','2014-12-25',
]

import csv
keys_csv = open('key.csv', 'rt')
keys=csv.reader(keys_csv)
store_station={}
for store,station in keys:
    store_station[store]=station

weather_csv = open('weather.csv', 'rt')
weather=csv.reader(weather_csv)
date_station_snowfall_rain={}
for w in weather:
    stationnumber=w[0]
    date=w[1]
    snowfall=w[13]
    rain=w[14]
    date_station_snowfall_rain[date+"_"+stationnumber]=snowfall+"_"+rain

train_csv = open('train.csv', 'rt')
train=csv.reader(train_csv)
train_date_store_item_units={}
count=0
for date,store_nbr,item_nbr,units in train:
       train_date_store_item_units[date+"_"+store_nbr+"_"+item_nbr]=units
        
train_csv1 = open('train.csv', 'rt')
train1=csv.reader(train_csv1)
train=[]
for t in train1:
    train.append(t)
    
test_csv = open('test.csv', 'rt')
test=csv.reader(test_csv)
test_date_store_item_units={}
count=0
for date,store_nbr,item_nbr in test:
       test_date_store_item_units[date+"_"+store_nbr+"_"+item_nbr]=units
        
#Final
date_store_station_item_quantity={}
def calculateResults(preds,dev_labels):
    correct, total = 0, 0
    for pred, label in zip(preds, dev_labels):
        if pred == label:
            correct += 1
        total += 1
   
    return correct/total

test_csv1 = open('test.csv', 'rt')
test1=csv.reader(test_csv1)
test=[]
for t in test1:
    test.append(t)

#Testing the score after adding in variables: Fridays, weekends, public holidays from 2012-2014

import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from random import shuffle

train_no_header = train[1:len(train)]
shuffle(train_no_header)

# add in day of the week variable where 0=Monday and 6=Sunday and every other day in between
train_new_variables = []
for x in train_no_header:
    year, month, day = (int(z) for z in x[0].split('-'))    
    day_of_week = datetime.date(year, month, day).weekday()
    day_of_year = datetime.date(year, month, day).timetuple().tm_yday
    month_of_year = datetime.date(year, month, day).month
    if day_of_week==4:
        is_friday=1
    else:
        is_friday=0
    if day_of_week in [4,5,6]:
        is_weekend=1
    else:
        is_weekend=0
    if x[0] in holidays:
        holiday_var=1
    else:
        holiday_var=0
    train_new_variables.append(x + [day_of_week] + [day_of_year] + [month_of_year] +[holiday_var] + [is_friday] + [is_weekend])
    
# our headers for reference
header = ['date', 'store_nbr', 'item_nbr', 'units', 'day_of_week', 'day_of_year', 'month_of_year',
         'holiday', 'is_friday', 'is_weekend']

train_columns = [1, 2, 6, 7, 9]

train_data = [[each_list[i] for i in train_columns] for each_list in train_new_variables]
train_labels = [train_new_variables[i][3] for i in range(0, len(train_new_variables))]

# test set
test_no_header = test[1:len(test)]

# add in day of the week variable where 0=Monday and 6=Sunday and every other day in between
test_new_variables = []
for x in test_no_header:
    year, month, day = (int(z) for z in x[0].split('-'))    
    day_of_week = datetime.date(year, month, day).weekday()
    day_of_year = datetime.date(year, month, day).timetuple().tm_yday
    month_of_year = datetime.date(year, month, day).month
    if day_of_week==4:
        is_friday=1
    else:
        is_friday=0
    if day_of_week in [4,5,6]:
        is_weekend=1
    else:
        is_weekend=0
    if x[0] in holidays:
        holiday_var=1
    else:
        holiday_var=0
    test_new_variables.append(x + [day_of_week] + [day_of_year] + [month_of_year] +[holiday_var] + [is_friday] + [is_weekend])
    
# our headers for reference
test_header = ['date', 'store_nbr', 'item_nbr', 'day_of_week', 'day_of_year', 'month_of_year',
         'holiday', 'is_friday', 'is_weekend']

test_columns = [1, 2, 5, 6, 8]

test_data = [[each_list[i] for i in test_columns] for each_list in test_new_variables]

# change data into floats
train_data = np.array(train_data).astype(np.float)
train_labels = np.array(train_labels).astype(np.float)
test_data = np.array(test_data).astype(np.float)

# Linear Regression using store_nbr, item_nbr, and day of the week
lr = LinearRegression()
linear_model = lr.fit(train_data, train_labels)
prediction = linear_model.predict(test_data)

print("initial predict done", time.time()-start_time)

#### prediction excluding zero units sold from data

# change labels from str to int
train_labels_int = [int(train_labels[i]) for i in range(0, len(train_labels))]

# turn data to pandas df
import pandas as pd
df = pd.DataFrame(
    {header[1] : [train_data[i][0] for i in range(0, len(train_data))],
    header[2]: [train_data[i][1] for i in range(0, len(train_data))],
    header[3]: train_labels_int},
    columns = [header[1], header[2], header[3]])

# sum of units sold grouped by store and item
units_sold_by_store_and_item = df.groupby([header[1], header[2]], as_index=False).sum()

#### look into changing this portion into other methods instead of list (set/dictionary) to make this run faster
# list of zero units sold using grouping of store and item
#zero_units_sold_list = []
#for i in range(0, len(units_sold_by_store_and_item)):
#    if units_sold_by_store_and_item['units'][i] == 0:
#        zero_units_sold_list += [[units_sold_by_store_and_item['store_nbr'][i], 
#                                  units_sold_by_store_and_item['item_nbr'][i]]]

#print("Zero units sold list created", time.time()-start_time)

sold_list = []
for i in range(0, len(units_sold_by_store_and_item)):
    if units_sold_by_store_and_item['units'][i] != 0:
        sold_list += [[units_sold_by_store_and_item['store_nbr'][i], 
                                  units_sold_by_store_and_item['item_nbr'][i]]]


# new training dataset called data_without_zeros containing no stores and items that sold zero units
data_without_zeros=[]
data_without_zeros=[]
for i in range(0, len(train_new_variables)):
    if [float(train_new_variables[i][1]), float(train_new_variables[i][2])] in sold_list:
        data_without_zeros += [train_new_variables[i]]
        if i == 100:
            print("100", time.time()-start_time)
        elif i == 10000:
            print("10k", time.time()-start_time)
        elif i == 50000:
            print("50k", time.time()-start_time)
        elif i == 100000:
            print("100k", time.time()-start_time)
        elif i == 150000:
            print("150k", time.time()-start_time)
        elif i == 500000:
            print("500k", time.time()-start_time)
        elif i == 1000000:
            print("1M", time.time()-start_time)

train_data_no_zeros = [[each_list[i] for i in train_columns] for each_list in data_without_zeros]
train_labels_no_zeros = [data_without_zeros[i][3] for i in range(0, len(data_without_zeros))]

print("training data without zeros done", time.time()-start_time)

# change data into floats
train_data_no_zeros = np.array(train_data_no_zeros).astype(np.float)
train_labels_no_zeros = np.array(train_labels_no_zeros).astype(np.float)

# Linear Regression using store_nbr, item_nbr, and day of the week
lr_no_zeros = LinearRegression()
linear_model_no_zeros = lr_no_zeros.fit(train_data_no_zeros, train_labels_no_zeros)
prediction_no_zeros = linear_model_no_zeros.predict(test_data)

# next test is to remove all 0 units sold
for i in range(0, len(prediction_no_zeros)):
    if prediction_no_zeros[i] + 1 < 0:
        prediction_no_zeros[i] = 0

print("prediction with negatives into 0 done", time.time()-start_time)

# if test data is in zero units sold list change prediction to 0
#prediction_final = []
#for i in range(0, len(test_data)):
#    if [test_data[i][1], test_data[i][2]] not in zero_units_sold_list:
#        prediction_final[i] += prediction_no_zeros[i]
#    else:
#        prediction_final[i] += 0
